Chapter 2 Exercises

# Erdos-Renyi Graphs

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
import random

from utils import decorate, savefig

# I set the random seed so the notebook 
# produces the same results every time.
np.random.seed(17)

# TODO: remove this when NetworkX is fixed
from warnings import simplefilter
import matplotlib.cbook
simplefilter("ignore", matplotlib.cbook.mplDeprecation)

## Exercises

In [ ]:
def all_pairs(nodes):
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if i < j:
                yield u, v

In [ ]:
def make_complete_graph(n):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(all_pairs(nodes))
    return G

In [ ]:
#From the original notebook
def reachable_nodes(G, start):
    seen = set()
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in seen:
            seen.add(node)
            stack.extend(G.neighbors(node))
    return seen

In [ ]:
def is_connected(G):
    start = next(iter(G))
    reachable = reachable_nodes(G, start)
    return len(reachable) == len(G)

**Exercise 2.4:** There are actually two kinds of ER graphs.  The one we generated in the chapter, $G(n, p)$, is characterized by two parameters, the number of nodes and the probability of an edge between nodes.

An alternative definition, denoted $G(n, m)$, is also characterized by two parameters: the number of nodes, $n$, and the number of edges, $m$.  Under this definition, the number of edges is fixed, but their location is random.

Repeat the experiments we did in this chapter using this alternative definition.  Here are a few suggestions for how to proceed:

1. Write a function called `m_pairs` that takes a list of nodes and the number of edges, $m$, and returns a random selection of $m$ edges.  A simple way to do that is to generate a list of all possible edges and use `random.sample`.

2. Write a function called `make_m_graph` that takes $n$ and $m$ and returns a random graph with $n$ nodes and $m$ edges.

3. Make a version of `prob_connected` that uses `make_m_graph` instead of `make_random_graph`.

4. Compute the probability of connectivity for a range of values of $m$.

How do the results of this experiment compare to the results using the first type of ER graph?

In [ ]:
def m_pairs(n,m):
    nodes = range(n)
    pairs = all_pairs(nodes)
    return random.sample(list(pairs),m) #choose m from all possible random edges

def make_m_graph(n,m):
    G = nx.Graph()
    G.add_nodes_from(list(range(n)))
    G.add_edges_from(m_pairs(n,m))
    
    return G

def prob_connected_m(n, m, iters=100):
    count = 0
    
    for i in range(iters):
        m_graph = make_m_graph(n, m)
        if is_connected(m_graph):
            count += 1
    
    return count/iters

m_graph = make_m_graph(10, 45)

nx.draw_circular(m_graph, node_color='C1', node_size=1000, with_labels=True)

In the book, m is related to n as follows for a complete graph:  

$$m = \frac{n(n-1)}{2}$$



In [ ]:
n = 10
p_star = np.log(n)/n
p_list = np.logspace(-1.3, 0, 11)

m_func = lambda n: n*(n-1)/2

m_list = [int(p*m_func(n)) for p in p_list] #get m values from list of probabilities

prob_connectivity_list = [prob_connected_m(n,m) for m in m_list ]



In [ ]:
plt.axvline(p_star, color='gray')
plt.plot(p_list, prob_connectivity_list, color='green')
plt.xscale('log')
plt.ylabel("Probability of connectivity")
plt.xlabel("Probability for an edge to connect ($p$)")
plt.xlim((p_list[0], p_list[-1]))

We see a similar trend as that in the earlier experiment, which is expected for an ER network. For $p < p^*$, the random network is unlikely to be connected. For  $p > p^*$, the random network is more likely to be connected.

In [ ]:
p_list